<a href="https://colab.research.google.com/github/project-mlx/mlx-projects-case-studies/blob/main/mobile_app_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mobile App Analytics
- user acquisitions
- user cohorts
- event periods
- Visualisations (WIP)

In [26]:
import pandas as pd
import numpy as np
import random

User acquisition class containing the required functions to calculate user acquisition, user cohort periods, and event periods.

In [80]:
class UserAcquisition:
  def __init__(self):
    pass
  def user_acquisition(self, df, event):
    """
     The function  for identifying the acquisition time for each user
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError('"dataset" needs to be a pandas dataframe')

    if not isinstance(event, str):
        raise TypeError('"event" needs to be a string')

    if event not in df['event'].unique():
        raise ValueError('"event" have to be a valid event present in the dataset')

    # get the acquisition time for each user
    acquisition = df[df['event'] == event].sort_values(
        'time').drop_duplicates(
            subset='user_id', keep='first')[['user_id', 'time']]

    # convert df to a dictionary
    acquisition = dict(zip(acquisition['user_id'], acquisition['time']))

    return acquisition

  def cohort_events_acquisition(self, df, event, period='week', month_format='period'):
    """
    The function to add "cohort", "event_period", "user_active" and "user_returns" columns.
    "cohort" is the weekly/monthly period that the user generated a successful plan (user acquired).
    "event_period" is the cohort that any event belongs in.
    "user_active" is True if the event took place at or after the user's acquisition time, False otherwise.
    "user_returns" is True if the event took place during a period subsequent to the acquisition cohort,
    False otherwise.
    """
    assert period in ['day', 'week', 'month'], '"period" should be either "day", "week" or "month"'

    if month_format:
        assert month_format in ['period', 'datetime'], '"month_format" should be either "period" or "datetime"'

    # user acquisition dictionary of unqiue acquired users
    acquisition = self.user_acquisition(df, event)
    users = acquisition.keys()

    # filter dataframe for only acquired users
    events = df[df['user_id'].isin(users)].copy()

    # get acquisition time for each user and create a "cohort" column
    events['acquisition_time'] = events['user_id'].map(acquisition)

    # create the "cohort" and "event_period" columns, based on the period defined
    if period == 'day':
        events['cohort'] = events['acquisition_time'].dt.date
        events['event_period'] = events['time'].dt.date

    elif period == 'week':
        events['cohort'] = (events['acquisition_time']
                            - events['acquisition_time'].dt.weekday.astype(
                                'timedelta64[D]')).astype('datetime64[D]')

        events['event_period'] = (events['time']
                                  - events['time'].dt.weekday.astype(
                                      'timedelta64[D]')).astype('datetime64[D]')

    else:
        # if monthly period, choose between pandas period type and datetime type
        # period type has a nice monthly format and is fine for aggregations
        # datetime would show up as first/last day of the month (yyyy-mm-dd)
        if month_format == 'period':
            events['cohort'] = events['acquisition_time'].dt.to_period('M')
            events['event_period'] = events['time'].dt.to_period('M')

        elif month_format == 'datetime':
            events['cohort'] = events['acquisition_time'].dt.date.astype('datetime64[M]')
            events['event_period'] = events['time'].dt.date.astype('datetime64[M]')

    # indicate if the user did any action at or after his/her acquisition time
    # if you do not want to count same-day activity replace following line with:
    # events['user_active'] = (events['time'].dt.date > events['acquisition_time'].dt.date)
    events['user_active'] = (events['time'] >= events['acquisition_time'])

    # indicate if the user returned in any period subsequent to his/her acquisition cohort
    events['user_returns'] = (events['event_period'] > events['cohort'])

    return events


  def users_per_period(self, df, event, user_category, period='week', month_format='period'):
    """
    The function to group new users into period cohorts.
    The first time a user generates a plan is treated as the acquisition time.
    """
    if user_category:
        assert hasattr(df, user_category), '"user_category" needs to be a column in the df dataset'

    # calculate the cohort for each user and period for each event
    events = self.cohort_events_acquisition(df, event, period=period, month_format=month_format)

    # will be used to rename the period column of each groupby result
    period_name = {'week': 'week_starting',
                  'month': "month"}

    # calculate size of each users cohort
    new_users = events.drop_duplicates(subset=['user_id', 'cohort']) \
        .groupby(['cohort']).size() \
        .reset_index() \
        .rename({0: 'new_users', 'cohort': period_name[period]}, axis=1) \
        .set_index(period_name[period])

    # break down new users into Organic/Non-organic
    if user_category:
        category = events[events['event'] == event] \
            .groupby(['cohort', 'user_category'])['user_id'].nunique() \
            .reset_index() \
            .rename({'user_id': 'new_users', 'cohort': period_name[period]}, axis=1) \
            .set_index(period_name[period])

        category = category.pivot(columns='user_category', values='new_users')[['organic', 'non-organic']] \
            .rename({'organic': 'new_organic_users', 'non-organic': 'new_non_organic_users'}, axis=1)

    # calculate number of active users per period
    active_users = events[events['user_active']] \
        .groupby(['event_period'])['user_id'].nunique() \
        .reset_index() \
        .rename({'user_id': 'active_users', 'event_period': period_name[period]}, axis=1) \
        .set_index(period_name[period])

    # calculate number of returning users per period
    returning_users = events[events['user_returns']] \
        .groupby(['event_period'])['user_id'].nunique() \
        .reset_index() \
        .rename({'user_id': 'returning_users', 'event_period': period_name[period]}, axis=1) \
        .set_index(period_name[period])

    # merge into a single dataframe
    if user_category:
        ds = new_users.join([category, active_users, returning_users], how='outer', sort=False).astype('Int64').copy()
    else:
        ds = new_users.join([active_users, returning_users], how='outer', sort=False).astype('Int64').copy()
    ds.fillna(0, inplace=True)

    # calculate period-on-period growth
    ds['w/w_growth'] = ds['new_users'].pct_change().apply(lambda x: "{0:.2f}%".format(x * 100))
    ds['new/return_ratio'] = (ds['new_users'] / ds['returning_users']) \
        .fillna(0) \
        .replace(np.inf, np.nan) \
        .apply(lambda x: "{0:.1f}".format(x))

    return ds


Generate dummy dataset

In [81]:
#empty dataset with 150k rows
df = pd.DataFrame({'user_id':0,
                   'user_category':None,
                   'event':None,
                   'time':pd.NaT},
                  index=list(range(150000)))

In [82]:
df.head()

,user_id,user_category,event,time
0,0,None,None,NaT
1,0,None,None,NaT
2,0,None,None,NaT
3,0,None,None,NaT
4,0,None,None,NaT


In [83]:
#generate 7k user_IDs randomly
user_ids = np.arange(7000)+1

#user categories: users who installed the app on their own are organic
#and the users who installed it through the campaign, advertisements 
#or for rewards, are non-organic users.
user_categories = ['organic','non-organic']

#create events list 
events = ['install','signup','click_other_content',
          'create_content','create_team','create_colab_content',
          'post_content','post_colab_content','delete_content']

#create a date range
dates = pd.date_range(start='2019-01-01',end='2020-12-31', freq='H')

In [84]:
#populate the generated values to the empty dataset
df.user_id = df.user_id.apply(lambda user: random.choice(user_ids))

#assign user category to each user randomly
user_cat_dict = {user_id:random.choice(user_categories) for user_id in df.user_id.unique()}
df.user_category = df.user_id.map(user_cat_dict)

#populate event and time columns 
#by randomly applying values from events and dates lists
df.event = df.event.apply(lambda event: random.choice(events))
df.time = df.time.apply(lambda time: random.choice(dates))

In [85]:
df.head()

,user_id,user_category,event,time
0,3637,non-organic,signup,2020-07-20 04:00:00
1,6078,organic,signup,2019-03-06 18:00:00
2,2982,organic,post_colab_content,2019-08-30 22:00:00
3,5446,organic,click_other_content,2020-08-12 12:00:00
4,6331,non-organic,click_other_content,2019-09-16 00:00:00


In [93]:
acquisition_class = UserAcquisition()

In [87]:
#Extracting acquisition time, user cohorts, events period
acquisition_class.cohort_events_acquisition(df=df,event='install').head()

,user_id,user_category,event,time,acquisition_time,cohort,event_period,user_active,user_returns
0,3637,non-organic,signup,2020-07-20 04:00:00,2019-02-23 04:00:00,2019-02-18,2020-07-20,True,True
1,6078,organic,signup,2019-03-06 18:00:00,2019-09-04 23:00:00,2019-09-02,2019-03-04,False,False
2,2982,organic,post_colab_content,2019-08-30 22:00:00,2019-01-15 22:00:00,2019-01-14,2019-08-26,True,True
3,5446,organic,click_other_content,2020-08-12 12:00:00,2020-01-30 22:00:00,2020-01-27,2020-08-10,True,True
4,6331,non-organic,click_other_content,2019-09-16 00:00:00,2019-07-10 02:00:00,2019-07-08,2019-09-16,True,True


In [88]:
#Activity statistics per period
acquisition_class.users_per_period(df=df,
                                   event='install',
                                   user_category='user_category',
                                   period='month')

,new_users,new_organic_users,new_non_organic_users,active_users,returning_users,w/w_growth,new/return_ratio
month,,,,,,,
2019-01,649,308,341,649,0,nan%,nan
2019-02,573,293,280,939,366,-11.71%,1.6
2019-03,557,276,281,1275,718,-2.79%,0.8
2019-04,531,260,271,1569,1038,-4.67%,0.5
2019-05,440,229,211,1819,1379,-17.14%,0.3
2019-06,394,202,192,1991,1597,-10.45%,0.2
2019-07,366,177,189,2257,1891,-7.11%,0.2
2019-08,335,161,174,2429,2094,-8.47%,0.2
2019-09,324,189,135,2575,2251,-3.28%,0.1
